In [98]:
import pandas as pd
import re
import numpy as np

peace1 = pd.read_csv("Data/tweets_with_hashtag_SayNoToWar.csv")
peace2 = pd.read_csv("Data/tweets2_with_hashtag_SayNoToWar.csv") 
peace3 = pd.read_csv("Data/tweets3_with_hashtag_SayNoToWar.csv") 
peace4 = pd.read_csv("Data/tweets4_with_hashtag_SayNoToWar.csv") 
war = pd.read_csv("Data/tweets_with_hashtag_FinalStrike.csv")

peace = peace1.iloc[:,1:].append(peace2.iloc[:,1:]).append(peace3.iloc[:,1:]).append(peace4.iloc[:,1:])

In [103]:
def get_mentioned_user(y):
    print(re.split(r'\s@(\w+):',y))
    return(0)

def get_mentions_sent(y):
    if len(y)>0:
        return(len(y)-1)
    else:
        return(0)
    
def get_content_type(y):
    if len(y)>0:
        return('RT')
    else:
        return('Tweet')
    
def find_location(df):
    df['country'] = np.where(df.location.notna(), df.location.apply(lambda x: str(x).split(',')[-1].strip().lower()),'No location')
    df['urdu'] = df.text.apply(lambda x: len(re.findall(r'[\u0600-\u06ff]+',x))>0)
    df.loc[df.urdu==1,'country'] = 'pakistan' 
    df['devanagari'] = df.text.apply(lambda x: len(re.findall(r'[\u0900-\u097f]+',x))>0)
    df.loc[df.devanagari==1,'country'] = 'india'
   
    df['pak_present'] = df.country.apply(lambda x: len(re.findall(r'pakistan',x))>0)
    df.loc[df.pak_present==1,'country'] = 'pakistan' 
    df['ind_present'] = df.country.apply(lambda x: len(re.findall(r'india',x))>0)
    df.loc[df.ind_present==1,'country'] = 'india'
    
    return df

def data_clean(df):
    df['retweeted_user']=df.text.apply(lambda x:''.join(re.findall(r'\s@(\w+):',x)))
    df['mentioned_user']= df.text.apply(lambda x:re.findall(r'\s@(\w+)',x))
    df['mentions_sent'] = df.mentioned_user.apply(get_mentions_sent)
    df['retweets_sent'] = df.retweeted_user.apply(lambda x: len(x))
    df['content_type'] = df.retweeted_user.apply(get_content_type)
    df['date'] = pd.to_datetime(df.created_at).apply(lambda x: x.date())
    df = find_location(df)
    df.drop(['ids','text','location','created_at','urdu','devanagari','pak_present','ind_present'],axis=1,inplace=True)
    return df

In [104]:
peace = data_clean(peace)
war = data_clean(war)

In [101]:
peace['country'].value_counts()

pakistan                          3824
No location                       2990
india                             1728
england                            185
پاکستان                            134
usa                                120
islamabad                          102
united kingdom                      80
lahore                              79
karachi                             78
canada                              72
united arab emirates                70
uk                                  60
ca                                  60
london                              58
chennai                             55
mumbai                              55
ontario                             48
भारत                                48
jammu and kashmir                   44
united states                       38
delhi                               37
bangalore                           36
ny                                  36
london/:lahore                      33
new delhi                

In [105]:
peace.head()

,retweet,favorite,inreplyto,friends,screen_name,followers,listed,retweeted_user,mentioned_user,mentions_sent,retweets_sent,content_type,date,country
0,38,0,NaN,561,imshee67,9924,18,TahirMujtaba_,[TahirMujtaba_],0,13,RT,2019-02-25,pakistan
1,1,0,NaN,24,SyedHas70245685,0,0,Shaistayousuff,[Shaistayousuff],0,14,RT,2019-02-25,pakistan
2,38,0,NaN,573,tabankhamosh,1064,11,TahirMujtaba_,[TahirMujtaba_],0,13,RT,2019-02-25,pakistan
3,1,5,NaN,155,Shaistayousuff,6295,4,,[],0,0,Tweet,2019-02-25,No location
4,0,1,NaN,293,AlvinaSajidK,808,5,,[],0,0,Tweet,2019-02-25,pakistan


In [107]:
peace.to_csv("Data/SayNoToWar.csv")
war.to_csv("Data/FinalStrike.csv")